# KAIST Summer Session 2018

## NAVER Movie Scraper for Downloading Actor Images (08.30.2018)

1. Generating the list of URLs to download
2. Downloading the webpage HTML from the URLs
3. Parsing the relevant information from the downloaded HTML

In [1]:
import requests
from bs4 import BeautifulSoup

### 1. Generating the list of URLs to download

1) Download the webpage HTML which includes the URL information

In [2]:
url_list = []

response = requests.get('https://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=cnt')
html = response.text
soup = BeautifulSoup(html, 'html.parser')
        
url_soup = soup.find_all('div', {'class':'tit3'})
for ul in url_soup:
    for a in ul.find_all('a'):
        print("Found the URL:", a['href'])
        url_list.append(a['href'])

print('Number of urls:',len(url_list))

Found the URL: /movie/bi/mi/basic.nhn?code=169015
Found the URL: /movie/bi/mi/basic.nhn?code=153687
Found the URL: /movie/bi/mi/basic.nhn?code=152661
Found the URL: /movie/bi/mi/basic.nhn?code=167697
Found the URL: /movie/bi/mi/basic.nhn?code=164115
Found the URL: /movie/bi/mi/basic.nhn?code=109906
Found the URL: /movie/bi/mi/basic.nhn?code=154222
Found the URL: /movie/bi/mi/basic.nhn?code=174626
Found the URL: /movie/bi/mi/basic.nhn?code=175322
Found the URL: /movie/bi/mi/basic.nhn?code=140652
Found the URL: /movie/bi/mi/basic.nhn?code=169643
Found the URL: /movie/bi/mi/basic.nhn?code=132996
Found the URL: /movie/bi/mi/basic.nhn?code=173019
Found the URL: /movie/bi/mi/basic.nhn?code=172010
Found the URL: /movie/bi/mi/basic.nhn?code=160721
Found the URL: /movie/bi/mi/basic.nhn?code=171499
Found the URL: /movie/bi/mi/basic.nhn?code=172425
Found the URL: /movie/bi/mi/basic.nhn?code=163533
Found the URL: /movie/bi/mi/basic.nhn?code=167602
Found the URL: /movie/bi/mi/basic.nhn?code=160375


### 2. Parsing the relevant information from the HTML

- Actor profile image parsing

In [3]:
import os
import urllib

movie_actor_collection = []

count = 0 

for movie_url in url_list:
    
    if count % 5 ==0:
        print(str(count),"/", len(url_list))    
        
    # download the html which includes the relevant information
    movie_actor_url= movie_url.replace('basic', 'detail')
    response = requests.get('https://movie.naver.com'+movie_actor_url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    
    # parsing for movie information
    movie_info = {}
    movie_info['url'] = 'https://movie.naver.com'+movie_url
    movie_info['ranking']=count+1
    movie_info['movie_id'] = movie_url.split('code=')[-1]
    
    a = soup.find('h3',{'class':'h_movie'})
    movie_info['title'] = a.find('a').string

    
    # creating a new folder
    if not(os.path.isdir('images')):
        os.makedirs('images')

    # parsing for actor information    
    a = soup.find_all('p',{'class':'p_thumb'})
    for i in range(len(a)):
        try:
            movie_info['actor_name',i] = a[i].find('a')['title']
            movie_info['actor_img',i] = a[i].find('img')['src']
            
            # download image files unless it is the default image
            if movie_info['actor_img',i] != 'https://ssl.pstatic.net/static/movie/2012/06/dft_img111x139.png':
                urllib.request.urlretrieve(movie_info['actor_img',i],'images\\'+movie_info['movie_id']+'_'+movie_info['actor_name',i]+'.jpg')
        except:
            pass
        
        
    count +=1
    movie_actor_collection.append(movie_info)

print("Done")

0 / 50
5 / 50
10 / 50
15 / 50
20 / 50
25 / 50
30 / 50
35 / 50
40 / 50
45 / 50
Done


- Saving the data as .xlsx

In [4]:
import pandas as pd

movie_actor_df = pd.DataFrame(movie_actor_collection)
movie_actor_df.to_excel('naver_movie_actor.xlsx')